In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import os
import cobra
import pickle

In [2]:

# Change the current working directory
os.chdir('/Users/alexisroberson/Metabolic_Modeling/Project')
os.getcwd()


'/Users/alexisroberson/Metabolic_Modeling/Project'

In [ ]:
data_dir2= Path.cwd()

Agora_data_dir2 = data_dir2/'AGORA_Models_for_Infant_gut'


Braken_Models_Meta_data_df2 = pd.read_csv('Dominant_species_Braken_name_w_Agora_models_taxID_df.csv')
Braken_Models_Meta_data_df2.drop_duplicates(subset=['taxid', 'Species', 'Model'], keep='first', inplace=True)
Clean_Braken_Agora_Model_df2 = Braken_Models_Meta_data_df2[Braken_Models_Meta_data_df2['Model']!='No Model']
Clean_Braken_Agora_Model_df2 = Clean_Braken_Agora_Model_df2.reset_index(drop=True)

Braken_data_df2 = pd.read_csv(data_dir2/'Data/SHINE_COPTR_Data/Full_data_copy.csv')
Braken_data_df2.drop(columns= ['Escherichiaphage vB_EcoS-Ro145c2YLVW','Escherichia phage vB_EcoS_fFiEco03','Escherichia phage 26','Escherichiaphage SZH-1'],inplace=True)



In [ ]:
Braken_data_df3 = pd.read_csv(data_dir2/'Data/SHINE_COPTR_Data/Full_data.csv')
Braken_data_df3.drop(columns= ['Escherichia phage vB_EcoS-Ro145c2YLVW','Escherichia phage vB_EcoS_fFiEco03','Escherichia phage 26','Escherichia phage SZH-1','Salmonella phage vB_SAg-RPN213'],inplace=True)
psuedocount = 0.1
Braken_data_df3 = Braken_data_df3.applymap(lambda x: psuedocount if x ==0 else x)



In [ ]:
#Clean_Braken_Agora_Model_df2
#Clean_Braken_Agora_Model_df2.drop_duplicates(subset=['taxid', 'Species'], keep='first', inplace=False)

df = pd.read_csv('Dominant_species_Braken_name_w_Agora_models_taxID_df.csv')
df['taxid'].nunique()
Species_Model_df = df.drop_duplicates(subset=['taxid'], keep='last')
Species_Model_df[Species_Model_df['Model'].duplicated()]

In [ ]:
Species_Model_df

In [ ]:
Single_model_Braken_Agora_df = Clean_Braken_Agora_Model_df2.drop_duplicates(subset=['taxid', 'Species'], keep='first', inplace=False)

In [ ]:
Single_model_Braken_Agora_df = Single_model_Braken_Agora_df.reset_index()
Single_model_Braken_Agora_df

### Using the clean Braken Speciess to Agora model (short version) based on Important_CreateBraken_AgoraModels_cleanedup Notebook (please use this data)

Keep in mind that we though we had 263 features but some where virus(missed) so I changed it in the Full_data_copy_df and redid our ML models since it had those features
Now we have 258 features

In [3]:
##Get Model dataframe 
#Get the data/Matrix
Single_model_Braken_Agora = pd.read_csv('Braken_Species_Agora_model_match__short_version_df.csv')
Single_model_Braken_Agora_df = pd.DataFrame(Single_model_Braken_Agora)

#get important info(taxid,species,Model) and get rid of 'no match part in species column
Single_model_Braken_Agora_df = Single_model_Braken_Agora_df[['taxid','Species','Model']]
Single_model_Braken_Agora_df['Species']=Single_model_Braken_Agora_df['Species'].str.replace('no match:','',case=False,regex=False) #clean up species name
Single_model_Braken_Agora_df



#Get Brakendata 



,taxid,Species,Model
0,100886,Catenibacterium mitsuokai,Catenibacterium_mitsuokai_DSM_15897
1,102684,Streptococcus infantarius,Streptococcus_infantarius_subsp_infantarius_AT...
2,103621,Actinomyces urogenitalis,Actinomyces_urogenitalis_DSM_15434
3,1115758,Romboutsia ilealis,Clostridium_dakarense_FF1
4,1134687,Klebsiella michiganensis,Klebsiella_michiganensis_10_5242
...,...,...,...
251,88431,Dorea longicatena,Dorea_longicatena_DSM_13814
252,89014,Blautia luti,Blautia_luti_DSM_14534
253,905,Acidaminococcus fermentans,Acidaminococcus_fermentans_DSM_20731
254,907,Megasphaera elsdenii,Megasphaera_elsdenii_DSM_20460


In [ ]:
Single_model_Braken_Agora_df[Single_model_Braken_Agora_df['Species']=='Faecalibacterium wellingii']

### Create Matrix X_B sample vs species (with a model)

In [4]:
#Create X_B matrix
data_dir2= Path.cwd()
Braken_data_df3 = pd.read_csv(data_dir2/'Data/SHINE_COPTR_Data/Full_data.csv')
Braken_data_df3.drop(columns= ['Escherichia phage vB_EcoS-Ro145c2YLVW','Escherichia phage vB_EcoS_fFiEco03','Escherichia phage 26','Escherichia phage SZH-1','Salmonella phage vB_SAg-RPN213',
                              'Lawsonella clevelandensis','Massilistercora timonensis'],inplace=True)
psuedocount = 0.1
Braken_data_df3 = Braken_data_df3.applymap(lambda x: psuedocount if x ==0 else x)




X_B = Braken_data_df3.iloc[:,3:-1] #This is our braken data (out of order for species)
X_B.rename(columns={'Escherichia coli_x': 'Escherichia coli','Butyrivibrio crossotus':'Eshraghiella crossota','Peptoniphilus sp. SAHP1':'Peptoniphilus genitalis','Faecalibacterium sp. HTF-F':'Faecalibacterium wellingii'}, inplace=True) # the taxid matches so we will change the name so we can have it in our matrix
X_B

#we now need to match the species to species with models
#Single_model_Braken_Agora_df_clean = Single_model_Braken_Agora_df.copy()
#Single_model_Braken_Agora_df_clean['Species']=Single_model_Braken_Agora_df_clean['Species'].str.replace('no match:','',case=False,regex=False) #clean up species name
#Single_model_Braken_Agora_df_clean_copy = Single_model_Braken_Agora_df_clean.copy()

species_list = list(Single_model_Braken_Agora_df['Species'])  #list of species with model with same order as our metabolic model dictionary list
#mask = X_B.columns.isin(species_list)
#false_columns = X_B.columns[~mask]
#X_B = X_B.drop(columns=false_columns)
#false_columns
## Get the species names from X_B columns
#x_b_species = X_B.columns.tolist()

## Filter Single_model_Braken_Agora_df_clean_copy to retain only rows with matching species in X_B
#Single_model_Braken_Agora_df_clean_filtered = Single_model_Braken_Agora_df_clean_copy[
    #Single_model_Braken_Agora_df_clean_copy['Species'].isin(x_b_species)
#]
#Single_model_Braken_Agora_df_clean_filtered = Single_model_Braken_Agora_df_clean_filtered.drop(columns='index')
correct_order_species_list = list(Single_model_Braken_Agora_df['Species'])

#reorder Xb to match braken_agora_model list order
X_B=X_B[correct_order_species_list] 
X_B

,Catenibacterium mitsuokai,Streptococcus infantarius,Actinomyces urogenitalis,Romboutsia ilealis,Klebsiella michiganensis,Ruminococcus bicirculans (ex Wegman et al. 2014),Coprococcus catus,Ruminococcus champanellensis,Megasphaera massiliensis,Leuconostoc lactis,...,Parabacteroides distasonis,Succinivibrio dextrinosolvens,Eggerthella lenta,Fusobacterium mortiferum,Faecalibacterium prausnitzii,Dorea longicatena,Blautia luti,Acidaminococcus fermentans,Megasphaera elsdenii,Limosilactobacillus mucosae
0,73.0,1439.0,0.1,400.0,0.1,21.0,31.0,0.1,0.1,0.1,...,78.0,26.0,0.1,81.0,64.0,100.0,28.0,0.1,10.0,41.0
1,36.0,23.0,39.0,49.0,31.0,0.1,0.1,10.0,1826.0,0.1,...,177.0,0.1,3688.0,0.1,115.0,33.0,17.0,129.0,72472.0,11.0
2,211.0,71.0,68796.0,33.0,38.0,23.0,82.0,15.0,25.0,0.1,...,164.0,15.0,1508.0,1096.0,199.0,2954.0,52.0,27.0,50.0,697.0
3,76.0,43.0,6894.0,13.0,51.0,17.0,160.0,20.0,316.0,0.1,...,836.0,0.1,2696.0,0.1,669.0,1479.0,252.0,30250.0,9658.0,10046.0
4,0.1,15.0,0.1,0.1,223.0,0.1,0.1,0.1,0.1,0.1,...,682.0,0.1,0.1,0.1,68.0,0.1,0.1,0.1,0.1,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749,0.1,107.0,0.1,187.0,194.0,0.1,31.0,0.1,23.0,15588.0,...,0.1,0.1,1070.0,0.1,221.0,279.0,76.0,0.1,44.0,34.0
750,50323.0,29.0,18.0,999.0,381.0,239.0,4170.0,154.0,3342.0,210.0,...,1730.0,29.0,2838.0,151.0,222366.0,21153.0,9337.0,416.0,181266.0,57.0
751,0.1,0.1,21.0,0.1,221.0,24.0,260.0,11.0,65.0,0.1,...,5702.0,0.1,11540.0,17.0,9462.0,39620.0,252.0,14.0,236.0,23.0
752,0.1,12.0,0.1,159.0,2899.0,0.1,0.1,0.1,90.0,0.1,...,0.1,0.1,0.1,28.0,26.0,41.0,28.0,11.0,1274.0,2570.0


### Load the models

In [5]:
Agora_data_dir2 = data_dir2/'AGORA_Models_for_Infant_gut'

# I am putting loaded models in a list so i have to initialize it 
loaded_models_short = []

# For is model in dataframe find path and load it 
for model_name2 in Single_model_Braken_Agora_df['Model']:
    # Construct the full path to the .mat file
    model_path2 = os.path.join(Agora_data_dir2, f"{model_name2}.mat")
    
    # Load the model and append it to the list
    try:
        model2 = cobra.io.load_matlab_model(model_path2)
        loaded_models_short.append(model2)
        print(f"Loaded model: {model_name2}")
    except FileNotFoundError:
        print(f"Model file not found: {model_path2}")
    except Exception as e:
        print(f"Error loading model {model_name2}: {e}")

# I will save the loaded models to a pickle file
pickle_file_path2 = "/Users/alexisroberson/Metabolic_Modeling/Project/loaded_models_short_use.pkl"  # Specify a valid path on VACC
with open(pickle_file_path2, 'wb') as my_file:
    pickle.dump(loaded_models_short, my_file)

print(f"Total loaded models saved: {len(loaded_models_short)}")



Set parameter Username
Academic license - for non-commercial use only - expires 2025-07-29


No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e
No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e


Loaded model: Catenibacterium_mitsuokai_DSM_15897


No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e


Loaded model: Streptococcus_infantarius_subsp_infantarius_ATCC_BAA_102


...

Loaded model: Blautia_luti_DSM_14534


No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, p


Loaded model: Acidaminococcus_fermentans_DSM_20731


No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e


Loaded model: Megasphaera_elsdenii_DSM_20460
Loaded model: Lactobacillus_mucosae_LM1
Total loaded models saved: 256


In [ ]:
# Save the loaded models to a pickle file
pickle_file_path2 = "/Users/alexisroberson/Metabolic_Modeling/Project/loaded_models_short_use.pkl"  # Specify a valid path on VACC
with open(pickle_file_path2, 'wb') as my_file:
    pickle.dump(loaded_models_short, my_file)

print(f"Total loaded models saved: {len(loaded_models_short)}")


In [6]:
loaded_models_short_dict = dict(zip(Single_model_Braken_Agora_df['Model'], loaded_models_short))
loaded_models_short_dict

{'Catenibacterium_mitsuokai_DSM_15897': <Model model at 0x2ec983950>,
 'Streptococcus_infantarius_subsp_infantarius_ATCC_BAA_102': <Model model at 0x1577b2ad0>,
 'Actinomyces_urogenitalis_DSM_15434': <Model model at 0x1575fa650>,
 'Clostridium_dakarense_FF1': <Model model at 0x1680d70d0>,
 'Klebsiella_michiganensis_10_5242': <Model model at 0x16a8740d0>,
 'Ruminococcus_bicirculans_80_3': <Model model at 0x15795e190>,
 'Coprococcus_catus_ERR2221258': <Model model at 0x36a39d050>,
 'Ruminococcus_champanellensis_18P13': <Model model at 0x168730890>,
 'Megasphaera_massiliensis_NP3': <Model model at 0x16b8bc2d0>,
 'Leuconostoc_argentinum_KCTC_3773': <Model model at 0x16cab4650>,
 'Providencia_alcalifaciens_Dmel2': <Model model at 0x17be4dd10>,
 'Staphylococcus_aureus_subsp_aureus_IS_250': <Model model at 0x17c0fc2d0>,
 'Alistipes_senegalensis_JC50': <Model model at 0x17d862290>,
 'Staphylococcus_hominis_SK119': <Model model at 0x17f46dad0>,
 'Streptococcus_salivarius_57_I': <Model model at 

### Get reactions list

In [7]:
def reactions_to_dataframe(model_list):
    # Initialize an empty DataFrame. This holds all reaction data
    df_final = pd.DataFrame()

    # Iterate through each model in the my model list
    for model in model_list:
        reaction_data = []

        # Collect reaction details from the current model
        for reaction in model.reactions:
            reaction_data.append({
                'reaction_id': reaction.id,
                'reaction_name': reaction.name,
                'reaction_equation': reaction.reaction
            })
        
        # Turn dictionary to a DataFrame for the current model
        df = pd.DataFrame(reaction_data)

        # Concatenate the new DataFrame to the final DataFrame ( we are putting together the matrix)
        df_final = pd.concat([df_final, df], ignore_index=True)

        # Group columns==> I dont want duplicated rxns after adding reactions from the current model
        df_final = df_final.groupby(['reaction_id', 'reaction_name', 'reaction_equation'], as_index=False).first()

    # Final grouping 
    df_final = df_final.groupby(['reaction_id', 'reaction_name', 'reaction_equation'], as_index=False).first()

    return df_final


In [8]:
#model_list=[model,model2]
reactionID_df = reactions_to_dataframe(loaded_models_short)
reactionID_df.columns
RxnID_only_df = reactionID_df.drop(['reaction_name', 'reaction_equation'],axis=1,inplace=False)

In [10]:
reactionID_df.to_csv('ReactionID_Name_Equation_fromShortList_DominantSPeciesModels.csv',index=False)

In [11]:
RxnID_only_df

,reaction_id
0,12DGR180tex
1,12DGR180ti
2,12DGR180tipp
3,12PPDSDH
4,12PPD_Stex
...,...
7526,tNCAPpp
7527,tNZP
7528,tNZPpp
7529,tSBZCOApp


### Get Matrix (Agora model x ReactionsID) Matrix A

In [12]:
model_list = list(Single_model_Braken_Agora_df['Model']) 
reaction_ids = list(RxnID_only_df['reaction_id']) 


# Initialize a DataFrame with zeros, using model names as the index and reaction IDs as columns
matrix = pd.DataFrame(0, index=loaded_models_short_dict.keys(), columns=reaction_ids)

# Populate the DataFrame
for model_name, model in loaded_models_short_dict.items():
    # Extract the reaction IDs from the model
    model_reactions = [reaction.id for reaction in model.reactions]  # Extract reaction IDs from the model object

    # Set values in the DataFrame to 1 where reaction IDs are present in the model
    for reaction_id in reaction_ids:
        if reaction_id in model_reactions:
            matrix.loc[model_name, reaction_id] = 1

# Display the resulting matrix
print(matrix)


                                                    12DGR180tex  12DGR180ti  \
Catenibacterium_mitsuokai_DSM_15897                           0           0   
Streptococcus_infantarius_subsp_infantarius_ATC...            0           0   
Actinomyces_urogenitalis_DSM_15434                            0           0   
Clostridium_dakarense_FF1                                     0           1   
Klebsiella_michiganensis_10_5242                              0           0   
...                                                         ...         ...   
Faecalibacterium_cf_prausnitzii_KLE1255                       0           0   
Dorea_longicatena_DSM_13814                                   0           0   
Blautia_luti_DSM_14534                                        0           0   
Acidaminococcus_fermentans_DSM_20731                          0           0   
Lactobacillus_mucosae_LM1                                     0           0   

                                                   

In [14]:
matrix.to_csv('A_matrix_AgoravsRxn_shortlist.csv')

In [13]:
matrix

,12DGR180tex,12DGR180ti,12DGR180tipp,12PPDSDH,12PPD_Stex,12PPDt,12PPDtpp,12aHSDHe,13DAMPPOX,13PPDH,...,tCZPpp,tDHNA,tDHNACOA,tDHNApp,tNCAP,tNCAPpp,tNZP,tNZPpp,tSBZCOApp,tSUCBZ
Catenibacterium_mitsuokai_DSM_15897,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,1,0,0,0
Streptococcus_infantarius_subsp_infantarius_ATCC_BAA_102,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
Actinomyces_urogenitalis_DSM_15434,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
Clostridium_dakarense_FF1,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Klebsiella_michiganensis_10_5242,0,0,0,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Faecalibacterium_cf_prausnitzii_KLE1255,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,1,0,0,0
Dorea_longicatena_DSM_13814,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
Blautia_luti_DSM_14534,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Acidaminococcus_fermentans_DSM_20731,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,1,0,0


In [ ]:
######Dont Runnn###### we already did it 
#Create X_B matrix

X_B = Braken_data_df3.iloc[:,3:-1] #This is our braken data (out of order for species)
X_B.rename(columns={'Escherichia coli_x': 'Escherichia coli'}, inplace=True) # the taxid matches so we will change the name so we can have it in our matrix


#we now need to match the species to species with models
Single_model_Braken_Agora_df_clean = Single_model_Braken_Agora_df.copy()
Single_model_Braken_Agora_df_clean['Species']=Single_model_Braken_Agora_df_clean['Species'].str.replace('no match:','',case=False,regex=False) #clean up species name
Single_model_Braken_Agora_df_clean_copy = Single_model_Braken_Agora_df_clean.copy()

species_list = list(Single_model_Braken_Agora_df_clean['Species'])  #list of species with model with same order as our metabolic model dictionary list
mask = X_B.columns.isin(species_list)
false_columns = X_B.columns[~mask]
X_B = X_B.drop(columns=false_columns)

# Get the species names from X_B columns
x_b_species = X_B.columns.tolist()

# Filter Single_model_Braken_Agora_df_clean_copy to retain only rows with matching species in X_B
Single_model_Braken_Agora_df_clean_filtered = Single_model_Braken_Agora_df_clean_copy[
    Single_model_Braken_Agora_df_clean_copy['Species'].isin(x_b_species)
]
Single_model_Braken_Agora_df_clean_filtered = Single_model_Braken_Agora_df_clean_filtered.drop(columns='index')
correct_order_species_list = list(Single_model_Braken_Agora_df_clean_filtered['Species'])

#reorder Xb to match braken_agora_model list order
X_B=X_B[correct_order_species_list] 


### Get Matrix B : species vs Agora models

In [16]:
# Initialize a DataFrame with zeros
Matrix_B = pd.DataFrame(0, index=correct_order_species_list, columns=loaded_models_short_dict.keys())

# Iterate over each species in the filtered DataFrame
for species in correct_order_species_list:
    # Check if the species has a corresponding model in the loaded_models_short_dict
    models_for_species = Single_model_Braken_Agora_df.loc[
        Single_model_Braken_Agora_df['Species'] == species,
        'Model'
    ]

    # Populate Matrix_B based on the presence of models
    for model in loaded_models_short_dict.keys():
        if model in models_for_species.values:  # Check if the model is in the list of models for this species
            Matrix_B.at[species, model] = 1  # Set to 1 if the model is found
        else:
            Matrix_B.at[species, model] = 0  # Set to 0 if the model is not found


Matrix_B

,Catenibacterium_mitsuokai_DSM_15897,Streptococcus_infantarius_subsp_infantarius_ATCC_BAA_102,Actinomyces_urogenitalis_DSM_15434,Clostridium_dakarense_FF1,Klebsiella_michiganensis_10_5242,Ruminococcus_bicirculans_80_3,Coprococcus_catus_ERR2221258,Ruminococcus_champanellensis_18P13,Megasphaera_massiliensis_NP3,Leuconostoc_argentinum_KCTC_3773,...,Bacteroides_uniformis_ATCC_8492,Bacteroides_vulgatus_ATCC_8482,Parabacteroides_distasonis_ATCC_8503,Succinivibrio_dextrinosolvens_H5,Fusobacterium_mortiferum_ATCC_9817,Faecalibacterium_cf_prausnitzii_KLE1255,Dorea_longicatena_DSM_13814,Blautia_luti_DSM_14534,Acidaminococcus_fermentans_DSM_20731,Lactobacillus_mucosae_LM1
Catenibacterium mitsuokai,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Streptococcus infantarius,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Actinomyces urogenitalis,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Romboutsia ilealis,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Klebsiella michiganensis,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Dorea longicatena,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
Blautia luti,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
Acidaminococcus fermentans,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
Megasphaera elsdenii,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Now lets do X_r = X_B*B*A

In [17]:
Matrix_A = matrix
Matrix_A.sum(axis=1)

Catenibacterium_mitsuokai_DSM_15897                          985
Streptococcus_infantarius_subsp_infantarius_ATCC_BAA_102    1009
Actinomyces_urogenitalis_DSM_15434                          1166
Clostridium_dakarense_FF1                                   1157
Klebsiella_michiganensis_10_5242                            1941
                                                            ... 
Faecalibacterium_cf_prausnitzii_KLE1255                     1890
Dorea_longicatena_DSM_13814                                 1044
Blautia_luti_DSM_14534                                      1217
Acidaminococcus_fermentans_DSM_20731                        1231
Lactobacillus_mucosae_LM1                                   1038
Length: 207, dtype: int64

In [ ]:
Matrix_B

In [ ]:
X_B

In [26]:
X_BB = X_B.dot(Matrix_B)
X_R = X_BB.dot(Matrix_A)
X_R

,12DGR180tex,12DGR180ti,12DGR180tipp,12PPDSDH,12PPD_Stex,12PPDt,12PPDtpp,12aHSDHe,13DAMPPOX,13PPDH,...,tCZPpp,tDHNA,tDHNACOA,tDHNApp,tNCAP,tNCAPpp,tNZP,tNZPpp,tSBZCOApp,tSUCBZ
0,104.2,57449.9,104.2,62045.7,1586.6,268865.6,1586.6,994.7,101.1,676.1,...,234048.2,36760.3,27.0,61.5,163348.2,234048.2,163348.2,234048.2,25.0,0.2
1,9110.3,99402.0,9110.3,11718.8,252519.3,2682227.6,252519.3,3165.4,363.1,1230.1,...,346259.9,147781.3,0.1,74645.2,217206.8,346259.9,217206.8,346259.9,0.1,220.0
2,12275.3,78221.3,12275.3,24009.4,17711.7,2851705.4,17711.7,17018.1,65.2,3594.1,...,95890.9,109245.3,29.0,182.3,232946.7,95890.9,232946.7,95890.9,0.1,71775.0
3,6705.1,106221.5,6705.1,175719.6,18032.2,2324700.4,18032.2,166515.4,47.1,376.1,...,54086.3,95671.4,0.1,10414.1,349265.2,54086.3,349265.2,54086.3,28.0,6894.1
4,3800.1,17932.5,3800.1,12575.3,831602.8,1081.2,831602.8,252.2,4707.0,6549.2,...,23661.2,1363.3,0.1,1235.3,19306.3,23661.2,19306.3,23661.2,0.1,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749,12.6,34511.1,12.6,49607.4,5726.7,4143116.4,5726.7,9702.7,188.1,3900.1,...,179889.4,61522.4,16.0,126.3,222722.7,179889.4,222722.7,179889.4,0.1,0.2
750,13647.0,127787.4,13647.0,46793.4,1387272.7,1513192.1,1387272.7,513665.2,652.1,1864.1,...,232819.6,852762.3,89.0,194358.0,671213.0,232819.6,671213.0,232819.6,900.0,41.0
751,3859.3,36754.2,3859.3,22594.6,59173.1,3146879.3,59173.1,344285.5,202.0,2237.1,...,40240.0,173020.3,37.0,529.1,424094.1,40240.0,424094.1,40240.0,0.1,21.1
752,0.7,263777.1,0.7,280769.1,19755.4,5517653.5,19755.4,980.8,2200.0,9128.0,...,103083.8,450946.3,14.0,1413.4,466356.8,103083.8,466356.8,103083.8,0.1,0.2


In [27]:
X_R_intermediate = pd.concat([Braken_data_df3.iloc[:,:3],X_R],axis=1)
X_R_final = pd.concat([X_R_intermediate,Braken_data_df3.iloc[:,-1:]],axis=1)
X_R_final

,run_accession,SubjectID_child,visitlabel,12DGR180tex,12DGR180ti,12DGR180tipp,12PPDSDH,12PPD_Stex,12PPDt,12PPDtpp,...,tDHNA,tDHNACOA,tDHNApp,tNCAP,tNCAPpp,tNZP,tNZPpp,tSBZCOApp,tSUCBZ,Ecoli_GR
0,ERR10901421,020-30471-1-7,M01,104.2,57449.9,104.2,62045.7,1586.6,268865.6,1586.6,...,36760.3,27.0,61.5,163348.2,234048.2,163348.2,234048.2,25.0,0.2,0.902055
1,ERR10901327,020-13118-1-6,M03,9110.3,99402.0,9110.3,11718.8,252519.3,2682227.6,252519.3,...,147781.3,0.1,74645.2,217206.8,346259.9,217206.8,346259.9,0.1,220.0,0.651277
2,ERR10901309,020-35873-1-4,M01,12275.3,78221.3,12275.3,24009.4,17711.7,2851705.4,17711.7,...,109245.3,29.0,182.3,232946.7,95890.9,232946.7,95890.9,0.1,71775.0,0.650389
3,ERR10901412,020-32940-1-4,M06,6705.1,106221.5,6705.1,175719.6,18032.2,2324700.4,18032.2,...,95671.4,0.1,10414.1,349265.2,54086.3,349265.2,54086.3,28.0,6894.1,0.484738
4,ERR10901419,020-03098-1-6,M01,3800.1,17932.5,3800.1,12575.3,831602.8,1081.2,831602.8,...,1363.3,0.1,1235.3,19306.3,23661.2,19306.3,23661.2,0.1,0.2,1.002545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749,ERR10900587,020-09304-1-2,M06,12.6,34511.1,12.6,49607.4,5726.7,4143116.4,5726.7,...,61522.4,16.0,126.3,222722.7,179889.4,222722.7,179889.4,0.1,0.2,0.610762
750,ERR10900589,020-36899-1-5,M18,13647.0,127787.4,13647.0,46793.4,1387272.7,1513192.1,1387272.7,...,852762.3,89.0,194358.0,671213.0,232819.6,671213.0,232819.6,900.0,41.0,0.494419
751,ERR10900578,020-10044-1-0,M12,3859.3,36754.2,3859.3,22594.6,59173.1,3146879.3,59173.1,...,173020.3,37.0,529.1,424094.1,40240.0,424094.1,40240.0,0.1,21.1,0.561884
752,ERR10900577,020-12089-1-5,M06,0.7,263777.1,0.7,280769.1,19755.4,5517653.5,19755.4,...,450946.3,14.0,1413.4,466356.8,103083.8,466356.8,103083.8,0.1,0.2,0.582671


In [28]:
X_R_final.to_csv('X_R_final.csv',index=False)

### Lets have some Fun (Add in the ML)

I now have Matrix of samplesxDominantSpecies and Matrix of SamplesxReactionsfrom
models that corresonded to species. 
Plan of Action:
   
   Approach 1: LightGBM has both lasso and ridge regulatization. I can see if I can throw all features at it with regularization to feature select.
  
   Approach 2: PCA or some type of clustering to reduce dimensions
   
   Approach 3: Use ML on rxn matrix to see which features where important and concat those to other data and use Lightgbm 

In [ ]:
Braken_data_df3

In [ ]:
Single_model_Braken_Agora_df